# Convert — Run a simple conversion of a PDB to the QDXF format

This notebook shows how to perform a simple conversion of PDBs to the QDXF format

# 0) Complete example
See the [sample notebook](/Quickstarts/convert-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import os
from glob import glob
from pathlib import Path

from pdbtools import pdb_fetch

import rush

## 1.1) Configuration

In [ ]:
# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "convert"]
WORK_DIR = Path.home() / "qdx" / "convert"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

In [ ]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)
YOUR_TOKEN = os.getenv("RUSH_TOKEN")

2024-03-21 10:15:55,191 - rush - INFO - Restoring by default via env


In [ ]:
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

## 1.2) Build your client

Initialize our rush client and fetch available module paths.

In [ ]:
# Get our client, for calling modules and using the rush API
os.environ["RUSH_TOKEN"] = YOUR_TOKEN

client = rush.build_blocking_provider_with_functions(
    workspace=WORK_DIR,
    batch_tags=TAGS,
)

2024-03-21 10:15:55,220 - rush - INFO - Restoring by default via env


# 2) Fetch PDBS

In [ ]:
pdb_ids = ["1B39", "1GIH"]
for pdb_id in pdb_ids:
    complex = list(pdb_fetch.fetch_structure(pdb_id))
    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb_id}.pdb", "w") as f:
        for line in complex:
            f.write(line)

# 3) Convert each PDB

In [ ]:
for pdb in glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")):
    (conformer,) = client.convert(
        "PDB", Path(pdb), tags=[Path(pdb).name], target="NIX"
    )
    print(conformer.download(overwrite=True).open().read()[0:50], "...")

2024-03-21 10:58:12,105 - rush - INFO - Trying to restore job with tags: ['1B39.pdb', 'qdx', 'rush-py-v2', 'demo', 'convert'] and path: github:talo/tengu-prelude/b345a0b0077225c63d904d0e03fb9ca1acec55ed#convert
2024-03-21 10:58:14,020 - rush - INFO - Argument a2e6241a-ea9b-4d46-a610-e8331c158552 is now ModuleInstanceStatus.RESOLVING
2024-03-21 10:58:15,139 - rush - INFO - Argument a2e6241a-ea9b-4d46-a610-e8331c158552 is now ModuleInstanceStatus.ADMITTED
2024-03-21 10:58:16,286 - rush - INFO - Argument a2e6241a-ea9b-4d46-a610-e8331c158552 is now ModuleInstanceStatus.RUNNING
2024-03-21 10:58:17,429 - rush - INFO - Argument a2e6241a-ea9b-4d46-a610-e8331c158552 is now ModuleInstanceStatus.AWAITING_UPLOAD
[{"amino_acid_insertion_codes": ["", "", "", "", " ...
2024-03-21 10:58:21,481 - rush - INFO - Trying to restore job with tags: ['1GIH.pdb', 'qdx', 'rush-py-v2', 'demo', 'convert'] and path: github:talo/tengu-prelude/b345a0b0077225c63d904d0e03fb9ca1acec55ed#convert
2024-03-21 10:58:21,783 

In [ ]:
client.status()